# Electrode Noise Detection

This notebook will implement a series of first pass machine learning algorithms to test rejection of electrodes.

Noise is categorized for us as the following:
1. EKG vs EEG
2. DC electrodes
3. Reference Electrodes
4. Band noise electrodes -> notch filter
5. General noise electrodes -> reject

Algorithms to systematically explore:
1. Linear Regression
2. Logistic Regression
3. LDA and QDA
4. Naive Bayes
5. KNN
6. SVM
7. LVQ
8. Random Forest
9. Adaboost